In [8]:

from database.sheetsorm.orm import SheetsORM
import os
import pandas as pd
SPREADSHEET_NAME  = os.getenv('SPREADSHEET_NAME','Pegasus - DEV')
USER_EMAIL = os.getenv('USER_EMAIL', 'pedrohenriquebraga735@gmail.com')

db = SheetsORM(scope=['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive'],
            credentials_file='database/credentials.json', spreadsheet_name=SPREADSHEET_NAME)
db.connect(create_if_not_exist=True)





KeyboardInterrupt: 

In [2]:
from database.entities import TBL_Category, TBL_Transactions
from datetime import datetime
a = TBL_Transactions(
    ID_Account = 1,
    CD_Type = 'Expense',
    DT_TransactionDate = datetime.now(),
    NR_Amount = .50,
    DS_Description = 'mercado'
)

a.__str__

<method-wrapper '__str__' of TBL_Transactions object at 0x0000014C3D3C05B0>

In [ ]:
repo.count(lambda x: x.DS_Name == 'teste')

13

In [ ]:
from database.sheetsorm.decorators import entity, column

@entity()
class TBL_Account:
    ID_Account = column(int, primary_key=True)

TBL_Account.ID_Account

In [ ]:
class Exemplo:
    nome : str  = 'teste'


a  = Exemplo()
a.nome  = 1

a.nome


1